# Model Pipeline

## Pipeline Setup

In [18]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [19]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()

In [20]:
local_path = "data/sensor_data.csv"

base_uri = f"s3://{default_bucket}/airdata"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-us-east-1-790237383528/airdata/sensor_data.csv


In [21]:
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

mse_threshold = ParameterFloat(name="MseThreshold", default_value=0.5)

## Preprocessing Script

In [31]:
%%writefile code/preprocessing.py
import argparse
import os
import requests
import tempfile

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.15
TEST_SPLIT = 0.15

base_dir = "/opt/ml/processing"

df_location = pd.read_csv(
    f"{base_dir}/input/sensor_data.csv"
)

# Split training
train_data = df_location.iloc[:int(len(df_location) * TRAIN_SPLIT)]
train_data = train_data.reset_index(drop=True)

# Split validation
val_data = df_location.iloc[int(len(df_location) * TRAIN_SPLIT):int(len(df_location) * TRAIN_SPLIT) + int(len(df_location) * VAL_SPLIT)]
val_data = val_data.reset_index(drop=True)

# Split testing
test_data = df_location.iloc[int(len(df_location) * TRAIN_SPLIT) + int(len(df_location) * VAL_SPLIT):int(len(df_location) * TRAIN_SPLIT) + int(len(df_location) * VAL_SPLIT) + int(len(df_location) * TEST_SPLIT)]
test_data = test_data.reset_index(drop=True)

# Normalize the training dataset
scaler = StandardScaler()
train_data.loc[:, "value"] = scaler.fit_transform(train_data["value"].values.reshape(-1, 1))
val_data.loc[:, "value"] = scaler.transform(val_data["value"].values.reshape(-1, 1))
test_data.loc[:, "value"] = scaler.transform(test_data["value"].values.reshape(-1, 1))

print("Train Data Shape:", train_data.shape)
print("Validation Data Shape:", val_data.shape)
print("Test Data Shape:", test_data.shape)

pd.DataFrame(train_data).to_csv(f"{base_dir}/train/train.csv")
pd.DataFrame(val_data).to_csv(f"{base_dir}/validation/validation.csv")
pd.DataFrame(test_data).to_csv(f"{base_dir}/test/test.csv")

Overwriting code/preprocessing.py


In [32]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-airdata-process",
    role=role,
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [33]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocessing.py",
)

step_process = ProcessingStep(name="AirDataProcess", step_args=processor_args)

In [34]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-790237383528/airdata/sensor_data.csv'},
  {'Name': 'MseThreshold', 'Type': 'Float', 'DefaultValue': 0.5}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'AirDataProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazona

## Model Training

## Model Evaluation

## Create Pipeline

In [35]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"AirDataPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        mse_threshold,
    ],
    steps=[step_process],#, step_train, step_eval, step_cond],
)

In [36]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-790237383528/airdata/sensor_data.csv'},
  {'Name': 'MseThreshold', 'Type': 'Float', 'DefaultValue': 0.5}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'AirDataProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazona

## Execute Pipeline

In [37]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()
try:
    execution.wait()
except Exception as error:
    print(error)

In [38]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:790237383528:pipeline/AirDataPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:790237383528:pipeline/AirDataPipeline/execution/3unk28l8sg75',
 'PipelineExecutionDisplayName': 'execution-1739734686920',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'airdatapipeline',
  'TrialName': '3unk28l8sg75'},
 'CreationTime': datetime.datetime(2025, 2, 16, 19, 38, 6, 865000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 16, 19, 40, 41, 139000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:790237383528:user-profile/d-y5kcordfnuyc/kellerflint',
  'UserProfileName': 'kellerflint',
  'DomainId': 'd-y5kcordfnuyc',
  'IamIdentity': {'Arn': 'arn:aws:sts::790237383528:assumed-role/LabRole/SageMaker',
   'PrincipalId': 'AROA3P7ORRNUBZEOQGRLH:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:790237383528:user-profile/d-y

In [39]:
execution.list_steps()

[{'StepName': 'AirDataProcess',
  'StartTime': datetime.datetime(2025, 2, 16, 19, 38, 7, 714000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 2, 16, 19, 40, 40, 620000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:790237383528:processing-job/pipelines-3unk28l8sg75-AirDataProcess-74vwPLyZlN'}},
  'AttemptCount': 1}]